In [3]:
from basic_vectorizer import TextVectorizer
import numpy as np
import pandas as pd
import torch
from torch import nn
import string
import re
import pickle

In [4]:
df1 = pd.read_csv('data/accountant/buh.csv')
df2 = pd.read_csv('data/ceo/busskapital.csv')
df3 = pd.read_csv('data/ceo/forbes_business.csv')
df4 = pd.read_csv('data/ceo/investnique.csv')
df5 = pd.read_csv('data/ceo/ppt_ru.csv')

In [5]:
df1

,Unnamed: 0,text,data
0,0,Редакция БУХ.1С напоминает о самых позитивных ...,2022-10-07 00:00:00
1,1,12 октября 2022 в 10:00 по местному времени фи...,2022-10-07 00:00:00
2,2,С 7 октября 2022 года вступили в силу поправки...,2022-10-07 00:00:00
3,3,"Эксперты Роструда разъяснили, вправе ли работо...",2022-10-07 00:00:00
4,4,На странице 1С:Лектория опубликована видеозапи...,2022-10-07 00:00:00
...,...,...,...
4466,4466,С 2021 года изменился порядок расчета НДФЛ с д...,2021-03-02 00:00:00
4467,4467,С 25 августа 2021 года срок аккредитации иност...,2021-03-02 00:00:00
4468,4468,"Эксперты ФНС уточнили в ходе вебинара, когда п...",2021-03-02 00:00:00
4469,4469,"Эксперты Роструда разъяснили, может ли работод...",2021-03-02 00:00:00


In [6]:
data = pd.concat([df1, df2, df3, df4, df5]).text
data

0       Редакция БУХ.1С напоминает о самых позитивных ...
1       12 октября 2022 в 10:00 по местному времени фи...
2       С 7 октября 2022 года вступили в силу поправки...
3       Эксперты Роструда разъяснили, вправе ли работо...
4       На странице 1С:Лектория опубликована видеозапи...
                              ...                        
1541    Изменение Воздушного кодекса РФ с 8 января 202...
1542    Генпрокуратура опубликовала сводный план прове...
1543    Изменение Уголовно-процессуального кодекса РФ ...
1544    Что изменится для самозанятых граждан в 2020 г...
1545    Повышение зарплат и пенсий военным в 2020 году...
Name: text, Length: 10269, dtype: object

In [13]:
topic_model = TextVectorizer()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alexzzmtsvv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexzzmtsvv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
class DatasetFixer:
    def __init__(self) -> None:
        pass

    def remove_punctuations(self, text):
        removed = text.translate(str.maketrans('', '', string.punctuation))
        return removed

    def remove_special_symbols(self, text):
        symbols = ["\n", "\r", "\xa0", "\u200b"]
        for symbol in symbols:
            text = text.replace(symbol, " ")

        return text
    
    def remove_extra_spaces(self, text):
        removed = re.sub(' +', ' ', text).strip()
        return removed
    
    def preprocess_description(self, text):
        preprocessed = text.strip().lower()
        preprocessed = self.remove_special_symbols(preprocessed)
        preprocessed = self.remove_punctuations(preprocessed)
        preprocessed = self.remove_extra_spaces(preprocessed)
        
        return preprocessed

In [15]:
cleaner = DatasetFixer()
train_data = []

for news in data.values:
    try:
        sample = topic_model.proccess_text(cleaner.preprocess_description(news))
        train_data.append(' '.join(sample))
    except:
        continue


In [8]:
topic_model.fit(train_data)

In [ ]:
topic_model.plot_top_words(20, 'Topics')

In [14]:
with open('count_vectorizer.pkl', 'wb') as f:
    pickle.dump(topic_model.count, f)

In [15]:
with open('latent_dirichlet_allocation.pkl', 'wb') as f:
    pickle.dump(topic_model.topic_model, f)

Делаем датасет в соответствии с отраслевыми лейблами

In [16]:
dataset = topic_model.make_labeled_dataset(train_data)

In [19]:
dataset[0][0]

'редакция бух 1 напоминать самый позитивный бухгалтерский новость уходить неделя минэкономразвития определить коэффициентыдефлятор 2023 год согласно проект приказ коэффициентдефлятор необходимый цель применение усн следующий год составить 1257 это значит 2023 год предельный величина доход применение упрощённый система налогообложение составить 18855 млн рубль 2514 млн рубль соответственно минфин предложить упростить правило лицензирование производитель продавец импортёр алкогольный спиртосодержащий продукция частность подготовить министерство поправка сокращать срок рассмотрение заявление соискатель лицензия осуществление деятельность производство оборот \u200e исключение розничный продажа этиловый спирт также законопроект сокращать перечень документ необходимый получение лицензия осуществление деятельность производство \u200e оборот спирт алкогольный \u200e спиртосодержащий продукция премьерминистр михаил мишустин подписать правительственный постановление согласно который 2023 год пла

In [21]:
df = pd.DataFrame(dataset, columns=['news', 'label'])
df.to_csv('industry_labeled.csv')

In [9]:
# бухгалтеры - df1
# руководители - df2-df5

df1['label'] = 0
df2['label'] = 1
df3['label'] = 1
df4['label'] = 1
df5['label'] = 1

In [25]:
labels = pd.concat([df1['label'], df2['label'], df3['label'], df4['label'], df5['label']]).values

labels

array([0, 0, 0, ..., 1, 1, 1])

In [26]:
dataset = [(text, label) for text, label in zip(train_data, labels)]

In [27]:
out = pd.DataFrame(dataset, columns=['news', 'label'])

In [29]:
out.to_csv('profession_labeled.csv')